# Basics of Numpyro for Bayesian Inference with MCMC 

In [7]:
import math
import os

import arviz as az
import matplotlib.pyplot as plt
import pandas as pd
from causalgraphicalmodels import CausalGraphicalModel
from IPython.display import Image, set_matplotlib_formats
from matplotlib.patches import Ellipse, transforms

import jax.numpy as jnp  # numpy, superfast
from jax import ops, random, vmap
from jax.scipy.special import expit

import numpy as onp       # the numpy, original

import numpyro as numpyro
import numpyro as npr
import numpyro.distributions as dist
from numpyro.diagnostics import effective_sample_size, print_summary
from numpyro.infer import MCMC, NUTS, Predictive

if "SVG" in os.environ:
    %config InlineBackend.figure_formats = ["svg"]
az.style.use("arviz-darkgrid")
numpyro.set_host_device_count(4)

## Distributions

In [8]:
b = dist.Bernoulli(0.3)

random_samples = b.sample(random.PRNGKey(0), (1000,))

In [9]:
with numpyro.handlers.seed(rng_seed=0):
    x = npr.sample('x', dist.Bernoulli(.3))

In [10]:
x

DeviceArray(1, dtype=int32)

In [11]:
x.item()

1

In [12]:
x * 100

DeviceArray(100, dtype=int32)

In [13]:
type(x)

jax.interpreters.xla._DeviceArray

In [18]:
with numpyro.handlers.seed(rng_seed=0):
    x = npr.sample('x', dist.Normal(0, jnp.array([1,2,4])))
x

DeviceArray([ 1.1378783, -2.44191  , -2.3661458], dtype=float32)